In [ ]:
from google.colab import files
files.upload()

In [19]:
import time
import datetime


class Tweet:
    def __init__(self, id, tweet_id, content, user_mentions_count, favorite_count, retweet_count, hashtag_id, user_id, created_at, sentiment, sentiment_score, location, latitude, longitude
):
        self.id = int(id)
        self.tweet_id = int(tweet_id)
        self.content = content
        self.user_mentions_count = user_mentions_count
        self.favorite_count = favorite_count
        self.retweet_count  = retweet_count
        self.hashtag_id = hashtag_id
        self.user_id = user_id
        self.created_at = datetime.datetime.strptime(created_at, "%Y-%m-%d %H:%M:%S.%f")
        self.sentiment  = sentiment
        self.sentiment_score = float(sentiment_score)
        self.location = location
        self.latitude = latitude
        self.longitude = longitude
        self.anamoly = 0.0
        self.x = 0.0
        self.y = 0.0
        self.length = len(content)

    def __str__(self):
        return self.created_at

In [20]:
import csv

id_vs_data = {}

with open('all_tweets_full_power.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='^')
    i = 0
    first = False
    for row in spamreader:
        if not first:
            print(', '.join(row))
            first = True
            continue
        row = [r for r in row]
        if len(row) != 14:
            print(', '.join(row))
            continue
        tweet = Tweet(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13])
        data = [r for r in row]
        id_vs_data[tweet.tweet_id] = tweet

print(len(id_vs_data))

1, 1579318298684297216, la patie girl power! non masterclass #avengersendgame, 0, 0, 0, 1, 1, 2022-10-10 12:14:06.003241, neutral, 0, Lincoln, 40.8257625, -96.6851982
442, 1538802937765408769, \as long as there are people who remember what is was, there will always be those that are unable to accept what it can be.\" ~ thanos (#avengersendgame) , ,  i mean, he might have been a twisted psychopath, but he wasn't really wrong.", 0, 0, 0, 1, 382, 2022-10-10 13:01:08.001569, neutral, 0, Toledo, 41.6639383, -83.5552120
1915, 1477061495649456132, today i popped in my bluray of #avengersendgame to have @chrisevans say \avengers assemble\" at midnight. and boom went the fireworks. makes my old heart smile, for just a while. also @brielarson's hands were so tiny compared to thanos' - almost cute. what a fun battle that was, , ^2", 0, 0, 1, 1592, 2022-10-10 11:14:37.003436, positive, 0.20000000298023224, Toledo, 41.6639383, -83.5552120
24868


In [21]:
max = 50

def check_topics(topics_map):
    if 1 not in topics_map or len(topics_map[1]) < max:
        return False
    if 2 not in topics_map or len(topics_map[2]) < max:
        return False
    if 3 not in topics_map or len(topics_map[3]) < max:
        return False
    if 4 not in topics_map or len(topics_map[4]) < max:
        return False
    if 0 not in topics_map or len(topics_map[0]) < max:
        return False
    return True

In [22]:
import csv

topics_map = {}
first = False
not_found = 0
found = 0

with open('final_topic.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
    i = 0

    for row in spamreader:
        if not first:
            print(', '.join(row))
            first = True
            continue
        if int(row[1]) not in id_vs_data:
            not_found = not_found + 1
            continue
        else:
            found = found + 1
        keywords = row[5]
        keywords = keywords.replace("{", "")
        keywords = keywords.replace("}", "")
        keywords = keywords.replace("'", "")
        keywords = keywords.replace("set()", "")
        if not keywords.strip():
            continue
        keywords = [keyword.strip() for keyword in keywords.split(",")]
        keywords.sort()
        data_store = (int(row[1]), int(row[4]), keywords)

        if int(row[4]) not in topics_map:
            topics_map[int(row[4])] = []
        topics_map[int(row[4])].append(data_store)

        anamoly = float(row[6])
        x = float(row[12])
        y = float(row[13])
        id_vs_data[int(row[1])].anamoly = anamoly
        id_vs_data[int(row[1])].x = x
        id_vs_data[int(row[1])].y = y
        if check_topics(topics_map):
            break

topics_map_trimmed = {}
for key, value in topics_map.items():
    print(key)
    print(value)
    topics_map_trimmed[key] = value[0:max]

topics_map = topics_map_trimmed

print(found)
print(not_found)

, id, text, lemma_tokens, topic_id, topic_keywords, score, all_scores_0, all_scores_1, all_scores_2, all_scores_3, all_scores_4, principal component 1, principal component 2
2
[(1579275857289482241, 2, ['avengersendgame', 'watch']), (1579271231366660096, 2, ['avengersendgame', 'watch']), (1579237922674704384, 2, ['avengersendgame', 'thor']), (1579232740662456320, 2, ['avengersendgame']), (1579231674889834496, 2, ['avengersendgame']), (1579231625527054337, 2, ['avengersendgame']), (1579230835748016128, 2, ['avengersendgame']), (1579230052478517248, 2, ['avengersendgame']), (1579229276813262849, 2, ['avengersendgame']), (1579229161335332864, 2, ['avengersendgame']), (1579228642257022976, 2, ['avengersendgame']), (1579228558064775168, 2, ['avengersendgame']), (1579228171635130370, 2, ['avengersendgame']), (1579227286955388928, 2, ['avengersendgame']), (1579227219443924993, 2, ['avengersendgame']), (1579226429304078337, 2, ['avengersendgame']), (1579224900459384833, 2, ['avengersendgame'])

In [23]:
def get_keywords_count(topics):
    name_vs_ids = {}
    all_words = set()
    for topic in topics:
        for word in topic[2]:
            all_words.add(word)
        combi = "::".join(topic[2])
        if combi not in name_vs_ids:
            name_vs_ids[combi] = set()
        name_vs_ids[combi].add(topic[0])
    return name_vs_ids, all_words

In [55]:
class Glyph:
    def __init__(self, name, start_time, end_time, user_volume, sentiment, anamoly, x, y):
        self.name = name
        self.start_time = start_time
        self.end_time = end_time
        self.user_volume = user_volume
        self.sentiment = sentiment
        self.anamoly  = anamoly
        self.x = x
        self.y = y

    def get_map(self):
        return {
            "name": self.name,
            "start_time": self.start_time,
            "end_time": self.end_time,
            "user_volume": self.user_volume,
            "sentiment": self.sentiment,
            "anamoly": self.anamoly,
            "x": self.x,
            "y": self.y
        };

    def __str__(self):
        return self.name

In [56]:
def get_glyph(name, children_keys):
    sentiment = 0
    for tweet_id in children_keys:
        if tweet_id not in id_vs_data:
            continue
        sentiment = sentiment + id_vs_data[tweet_id].sentiment_score
    sentiment = sentiment / len(children_keys)

    anamoly = 0
    for tweet_id in children_keys:
        if tweet_id not in id_vs_data:
            continue
        anamoly = anamoly + id_vs_data[tweet_id].anamoly
    anamoly = anamoly / len(children_keys)

    x = 0
    for tweet_id in children_keys:
        if tweet_id not in id_vs_data:
            continue
        x = x + id_vs_data[tweet_id].x
    x = x / len(children_keys)
    print(x)

    y = 0
    for tweet_id in children_keys:
        if tweet_id not in id_vs_data:
            continue
        y = y + id_vs_data[tweet_id].y
    y = y / len(children_keys)
    print(y)
    print('******')

    start_time = None
    for tweet_id in children_keys:
        if tweet_id not in id_vs_data:
            continue
        if not start_time or start_time > id_vs_data[tweet_id].created_at:
            start_time = id_vs_data[tweet_id].created_at

    end_time = None
    for tweet_id in children_keys:
        if tweet_id not in id_vs_data:
            continue
        if not end_time or end_time < id_vs_data[tweet_id].created_at:
            end_time = id_vs_data[tweet_id].created_at

    # print(sentiment)
    # print(anamoly)
    # print(x)
    # print(y)
    # print(start_time)
    # print(end_time)
    # print("***")
    glyph = Glyph(name, start_time, end_time, len(children_keys), sentiment, anamoly, x, y).get_map()

    return glyph

In [57]:
def get_children(name_vs_ids, names, all_words, length):
    parent_vs_child = []
    all_keys = set()
    for name in names:
        keyword_words = name.split("::")
        if len(keyword_words) == length:
            if length == 1:
                children = {}
                children_keys = name_vs_ids.get(keyword_words[0])
                glyph = get_glyph(name, children_keys)
                # parent_vs_child.append({"name": name,
                parent_vs_child.append({"name": "",
                                        "glyph": glyph,
                                        "childrens":list(children_keys),
                                        "value": len(children_keys)})
            else:
                all_keys = all_keys.union(name_vs_ids.get(name))
                children, children_keys = get_children(name_vs_ids, names, keyword_words, length - 1)
                glyph = get_glyph("{}:{}".format(name, len(children_keys)), children_keys)
                # if(len(children_keys) < 400):
                #     parent_vs_child.append({"name": name,
                #                         "glyph": glyph,
                #                         "childrens":list(children_keys),
                #                         "value": len(children_keys)})
                # else:
                # parent_vs_child.append({"name": "{}:{}".format(name, len(children_keys)),
                parent_vs_child.append({"name": "",
                                        "glyph": glyph,
                                        "children": children,
                                        "childrens":list(children_keys),
                                        "value": len(children_keys)})
            all_keys = all_keys.union(children_keys)
    return parent_vs_child, all_keys

In [58]:
full_list = []
for i in range(0, 5):
    topics = topics_map[i]
    name_vs_ids, all_words = get_keywords_count(topics)
    print(name_vs_ids)
    children, children_keys = get_children(name_vs_ids, list(name_vs_ids.keys()), all_words, len(all_words) - 1)
    parent_vs_children = {"children": children, "name": "", "childrens": list(children_keys)}
    full_list.append(parent_vs_children)

final_data = {"children": full_list, "name": 'all'}

{'avengersendgame': {1579229743576100864, 1579229752220868609, 1579229280110014466, 1579228219097907200, 1579227727466729472, 1579227610399113216, 1579227344215998464, 1579226486716116993, 1579226477157310465, 1579226430646677504, 1579225019900891136, 1579224704967389185, 1579208431508258816, 1579207306667229184, 1579204444813918209, 1573245892525969410, 1568087121553940482, 1565191340488531968, 1564041794505117699, 1566479642265272320, 1564040313445957632, 1571330753702551552, 1567108215787356161, 1565028385834803206, 1563994113061576704, 1561774742465441792, 1563236159471652865, 1566640029522591744, 1579158518258577408, 1572818282746462208, 1571908275909672960, 1564394773628305408}, 'movie': {1577195705793662976, 1577171858465886209, 1576989264373764096, 1576963184103411713, 1577150615339163648, 1576931217043165185, 1576896032000004096, 1576884740065411072}, 'm::movie': {1576884425131819008}, 'avengersendgame::year': {1576129360708308992, 1568116933328834561}, 'avengersendgame::movie

In [59]:
all_glyphs = []
for i in range(0, 5):
    topics = topics_map[i]
    name_vs_ids, all_words = get_keywords_count(topics)
    for name, ids in name_vs_ids.items():
        print(len(ids))
        all_glyphs.append(get_glyph(name, ids))

xs = set()
ys = set()
for glyph in all_glyphs:
    xs.add(glyph['x'])
    ys.add(glyph['y'])
print(xs)
print(ys)

32
-0.5865749034785396
0.6351509590248694
******
8
-0.7920026638826498
1.0459329310477161
******
1
-0.7931538815267741
1.047496827455013
******
2
-0.7818449363519644
1.0374889996794385
******
1
-0.7785154880217627
1.0363133335189958
******
1
-1.1386010585446311
0.3152809418035075
******
5
-0.49081833697952354
1.018589926429343
******
16
0.8375443412863588
-0.8667075613187708
******
5
1.258763981377859
-0.7514921408360861
******
21
1.0039405221819833
-0.5378264604325211
******
2
0.8813182665627466
-0.9272195418004782
******
1
1.082777566105453
-0.47343755951440164
******
1
1.9654149030371013
-1.2095049790926686
******
3
0.710551233546127
-0.8561234678989158
******
1
1.4588338829453795
-0.09129746826885644
******
3
-1.570837832012949
-1.402680017045049
******
6
-1.507191868203522
-0.9004041098103474
******
38
-1.5025688658299579
-1.0967654780428233
******
2
-0.6037283696218491
-0.6963753979435565
******
1
-1.065875059705323
-1.546480096345044
******
11
-0.1305273852188704
0.7974186919439

In [60]:
len(all_glyphs)

46

In [61]:
import json

with open("cluster_view_data.json", "w") as outfile:
    json.dump(final_data, outfile, default=str)

In [62]:
with open("mds_view_data.json", "w") as outfile:
    json.dump(all_glyphs, outfile, default=str)

In [63]:
topics_map

{2: [(1579275857289482241, 2, ['avengersendgame', 'watch']),
  (1579271231366660096, 2, ['avengersendgame', 'watch']),
  (1579237922674704384, 2, ['avengersendgame', 'thor']),
  (1579232740662456320, 2, ['avengersendgame']),
  (1579231674889834496, 2, ['avengersendgame']),
  (1579231625527054337, 2, ['avengersendgame']),
  (1579230835748016128, 2, ['avengersendgame']),
  (1579230052478517248, 2, ['avengersendgame']),
  (1579229276813262849, 2, ['avengersendgame']),
  (1579229161335332864, 2, ['avengersendgame']),
  (1579228642257022976, 2, ['avengersendgame']),
  (1579228558064775168, 2, ['avengersendgame']),
  (1579228171635130370, 2, ['avengersendgame']),
  (1579227286955388928, 2, ['avengersendgame']),
  (1579227219443924993, 2, ['avengersendgame']),
  (1579226429304078337, 2, ['avengersendgame']),
  (1579224900459384833, 2, ['avengersendgame']),
  (1579224708683567104, 2, ['avengersendgame']),
  (1579222686521511939, 2, ['avengersendgame']),
  (1579221117373669376, 2, ['avengersend

In [64]:
"""import time
import datetime


class Tweet:
    def __init__(self, id, tweet_id, content, user_mentions_count, favorite_count, retweet_count, hashtag_id, user_id, created_at, sentiment, sentiment_score, location, latitude, longitude
):
        self.id = int(id)
        self.tweet_id = int(tweet_id)
        self.content = content
        self.user_mentions_count = user_mentions_count
        self.favorite_count = favorite_count
        self.retweet_count  = retweet_count
        self.hashtag_id = hashtag_id
        self.user_id = user_id
        self.created_at = datetime.datetime.strptime(created_at, "%Y-%m-%d %H:%M:%S.%f")
        self.sentiment  = sentiment
        self.sentiment_score = float(sentiment_score)
        self.location = location
        self.latitude = latitude
        self.longitude = longitude
        self.anamoly = 0.0
        self.x = 0.0
        self.y = 0.0
        self.length = len(content)

    def __str__(self):
        return self.created_at"""

def get_tweet_json(tweet):
    tweet_json = {}
    tweet_json['id'] = tweet.id
    tweet_json['tweet_id'] = tweet.tweet_id
    tweet_json['content'] = tweet.content
    tweet_json['user_mentions_count'] = tweet.user_mentions_count
    tweet_json['favorite_count'] = tweet.favorite_count
    tweet_json['retweet_count'] = tweet.retweet_count
    tweet_json['hashtag_id'] = tweet.hashtag_id
    tweet_json['user_id'] = tweet.user_id
    tweet_json['created_at'] = tweet.created_at.strftime("%Y-%m-%d %H:%M:%S.%f")
    tweet_json['sentiment'] = tweet.sentiment
    tweet_json['sentiment_score'] = tweet.sentiment_score
    tweet_json['location'] = tweet.location
    tweet_json['latitude'] = tweet.latitude
    tweet_json['longitude'] = tweet.longitude
    tweet_json['anamoly'] = tweet.anamoly
    tweet_json['x'] = tweet.x
    tweet_json['y'] = tweet.y
    tweet_json['length'] = tweet.length
    return tweet_json

get_tweet_json(id_vs_data[1579275857289482241])

{'id': 3,
 'tweet_id': 1579275857289482241,
 'content': 'just watched #avengersendgame yet again and cried just as hard at the end as i have the first time i saw it. #everytime',
 'user_mentions_count': '0',
 'favorite_count': '0',
 'retweet_count': '0',
 'hashtag_id': '1',
 'user_id': '3',
 'created_at': '2022-10-10 13:10:24.006041',
 'sentiment': 'positive',
 'sentiment_score': 0.10000000149011612,
 'location': 'Seattle',
 'latitude': '47.6062095',
 'longitude': '-122.3320708',
 'anamoly': 0.9095846,
 'x': -1.8690203238715988,
 'y': -1.4898624046675562,
 'length': 119}

In [65]:
full_tweets_final = {}

for key, value in topics_map.items():
    for (tweet_id, _, _) in value:
        full_tweets_final[tweet_id] = get_tweet_json(id_vs_data[tweet_id])
print(full_tweets_final)

{1579275857289482241: {'id': 3, 'tweet_id': 1579275857289482241, 'content': 'just watched #avengersendgame yet again and cried just as hard at the end as i have the first time i saw it. #everytime', 'user_mentions_count': '0', 'favorite_count': '0', 'retweet_count': '0', 'hashtag_id': '1', 'user_id': '3', 'created_at': '2022-10-10 13:10:24.006041', 'sentiment': 'positive', 'sentiment_score': 0.10000000149011612, 'location': 'Seattle', 'latitude': '47.6062095', 'longitude': '-122.3320708', 'anamoly': 0.9095846, 'x': -1.8690203238715988, 'y': -1.4898624046675562, 'length': 119}, 1579271231366660096: {'id': 4, 'tweet_id': 1579271231366660096, 'content': 'just watched the tony stark death scene in endgame for the 57,00th time and it still gets very dusty. #avengersendgame #tonystark #mcu', 'user_mentions_count': '0', 'favorite_count': '0', 'retweet_count': '0', 'hashtag_id': '1', 'user_id': '4', 'created_at': '2022-10-10 13:27:21.006521', 'sentiment': 'negative', 'sentiment_score': -0.2000

In [66]:
with open("full_tweets_data.json", "w") as outfile:
    json.dump(full_tweets_final, outfile, default=str)

In [ ]:
def print_below(parent_vs_children, stars):
    # print(len(parent_vs_children['value']))
    print(stars)
    for value in parent_vs_children['children']:
        print(value['name'])
        print_below(value['children'], stars + stars)

In [ ]:
parent_vs_children

{'children': [{'name': 'avengersendgame::movie::watch:56',
   'children': [{'name': 'avengersendgame::watch:45',
     'children': [{'name': 'avengersendgame', 'value': 45}],
     'value': 45},
    {'name': 'avengersendgame::thor:45',
     'children': [{'name': 'avengersendgame', 'value': 45}],
     'value': 45},
    {'name': 'avengersendgame::movie:45',
     'children': [{'name': 'avengersendgame', 'value': 45}],
     'value': 45}],
   'value': 56}],
 'name': '4'}

In [ ]:
print_below(parent_vs_children, "****")

57
****
avengersendgame::movie::watch
56
********
avengersendgame::watch
45
****************
avengersendgame
45
********************************
avengersendgame::thor
45
****************
avengersendgame
45
********************************
avengersendgame::movie
45
****************
avengersendgame
45
********************************


In [ ]:
topic_keywords = set()

for topic in topics:
    topic_keywords.add("::".join(topic[2]))

topic_keywords

{'avengersendgame',
 'avengersendgame::movie',
 'avengersendgame::movie::watch',
 'avengersendgame::watch',
 'thor::avengersendgame'}

In [ ]:
triple_or_more = {}

for topic in topics:
    if len(topic[2]) >= 3:
        triple_or_more["::".join(topic[2])] = topic[2]

triple_or_more_extra = {}

for key, data in triple_or_more.items():
    data = list(data)
    lists = []
    for i in range(len(data) + 1):
        for j in range(i):
            sublist = data[j: i]
            if(len(sublist) == 2):
                lists.append("::".join(sublist))
    triple_or_more_extra[key] = [triple_or_more[key], lists, [0]*len(lists)]

triple_or_more = triple_or_more_extra

print(triple_or_more)

{'avengersendgame::movie::watch': [{'avengersendgame', 'movie', 'watch'}, ['avengersendgame::movie', 'movie::watch'], [0, 0]]}


In [ ]:
double = {}

for topic in topics:
    if len(topic[2]) == 2:
        combined = "::".join(topic[2])
        for key, value in triple_or_more.items():
            for idx, combi in enumerate(value[1]):
                if combined == combi:
                    value[2][idx] = value[2][idx] + 1
        double["::".join(topic[2])] = topic[2]

double_extra = {}

for key, data in double.items():
    data = list(data)
    lists = []
    for i in range(len(data) + 1):
        for j in range(i):
            sublist = data[j: i]
            if(len(sublist) == 1):
                lists.append("::".join(sublist))
    double_extra[key] = [double[key], lists, [0]*len(lists)]

double = double_extra

print(double)
print(triple_or_more)

{'avengersendgame::watch': [{'avengersendgame', 'watch'}, ['avengersendgame', 'watch'], [0, 0]], 'thor::avengersendgame': [{'thor', 'avengersendgame'}, ['thor', 'avengersendgame'], [0, 0]], 'avengersendgame::movie': [{'avengersendgame', 'movie'}, ['avengersendgame', 'movie'], [0, 0]]}
{'avengersendgame::movie::watch': [{'avengersendgame', 'movie', 'watch'}, ['avengersendgame::movie', 'movie::watch'], [2, 0]]}


In [ ]:
ones = {}

for topic in topics:
    if len(topic[2]) == 1:
        combined = "::".join(topic[2])
        for key, value in double.items():
            for idx, combi in enumerate(value[1]):
                if combined == combi:
                    value[2][idx] = value[2][idx] + 1
        ones["::".join(topic[2])] = topic[2]

print(ones)
print(double)

{'avengersendgame': {'avengersendgame'}}
{'avengersendgame::watch': [{'avengersendgame', 'watch'}, ['avengersendgame', 'watch'], [38, 0]], 'thor::avengersendgame': [{'thor', 'avengersendgame'}, ['thor', 'avengersendgame'], [0, 38]], 'avengersendgame::movie': [{'avengersendgame', 'movie'}, ['avengersendgame', 'movie'], [38, 0]]}
